In [ ]:
import base64
import requests
import re
import json
import pandas as pd
import os
import yaml

In [ ]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [ ]:

prompt = """Actúa como observador de colectas que encuentra y extrae la información solicitada de una imagen de un informe de colecta.
Necesito que extraigas TODA la información específica donde encuentres altitud, latitud y longitud y la estructures en formato JSON que contenga todas las accesiones diferentes
relacionadas en esa colección y asignar a cada una la información correspondiente de: colector, número de accesión, país, departamento, estatus, especie, nombre local, municipio, fuente, fecha, lugar, altitud, longitud, latitud. 
Puede haber más información o menos información que la mencionada. El objetivo que no debes dejar pasar es el identificador de la semilla, altitud, longitud y latitud en TODO.

Este es un ejemplo de como deberia resultar la data de una accesión:

    "colectores": "DEBOUCK DG LOPEZ L ANGULO N & TORO O",
    "numero_accesión": 2928,
    "pais": "CLB",
    "departamento": "HUILA",
    "municipio": "SANTA MARIA",
    "especie": "P.lunatus",
    "lugar": "7 Km W Puerto El Socorro, Las Delicias",
    "altitud": 1030,
    "longitud": "75.34W",
    "latitud": "02.57N"

Aquí está la imagen que necesito que procese"""

In [ ]:
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')
  
def stringToJson(text):
    json_match = re.search(r'```json\n(.*\n)```', text, re.DOTALL)
    if json_match:
        json_str = json_match.group(1).strip()
        try:
            # Carga el JSON desde la cadena
            json_data = json.loads(json_str)
            return json_data
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")
            return None
    else:
        print("No JSON found in the string.")
        return None

def extract_json_from_string(string):
    # Define una expresión regular para encontrar el JSON dentro de la cadena
    json_pattern = re.compile(r'\{.*?\}', re.DOTALL)
    
    # Busca el JSON en la cadena
    match = json_pattern.search(string)
    
    if match:
        json_str = match.group(0)
        try:
            # Intenta convertir la cadena JSON a un objeto Python
            json_obj = json.loads(json_str)
            return json_obj
        except json.JSONDecodeError:
            raise ValueError("La cadena contiene un JSON no válido")
    else:
        raise ValueError("No se encontró un JSON en la cadena proporcionada")

In [ ]:
# Path
directory_path = '/home/ec2-user/SageMaker/genebanks/data/2_images/Germplasm collection-Colombia-1990'
# Crear un DataFrame vacío
df = pd.DataFrame()

In [ ]:
# Recorrer todas las imágenes en el directorio
for filename in os.listdir(directory_path):
    if filename.endswith(".png") or filename.endswith(".jpg") or filename.endswith(".jpeg"):
        image_path = os.path.join(directory_path, filename)

        # Codificar la imagen
        base64_image = encode_image(image_path)

        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {api_key}"
        }

        payload = {
            "model": "gpt-4o",
            "messages": [
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": f"{prompt}"
                        },
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/jpeg;base64,{base64_image}"
                            }
                        }
                    ]
                }
            ],
            "max_tokens": 1200
        }

        response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
        res = response.json()

        try:
            extracted_json = extract_json_from_string(res['choices'][0]['message']['content'])
            if extracted_json:
                # Agregar el JSON extraído al DataFrame
                df = pd.concat([df, pd.DataFrame([extracted_json])], ignore_index=True)
        except ValueError as e:
            print(e)

# Mostrar el DataFrame resultante


In [ ]:
# Getting the base64 string
base64_image = encode_image(image_path)

headers = {
  "Content-Type": "application/json",
  "Authorization": f"Bearer {api_key}"
}

payload = {
  "model": "gpt-4o",
  "messages": [
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": f"{prompt}"
        },
        {
          "type": "image_url",
          "image_url": {
            "url": f"data:image/jpeg;base64,{base64_image}"
          }
        }
      ]
    }
  ],
  "max_tokens": 1200
}

In [ ]:
response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

In [ ]:
res = response.json()

In [ ]:
print(res)

In [ ]:
def extract_json_from_string(string):
    # Define una expresión regular para encontrar el JSON dentro de la cadena
    json_pattern = re.compile(r'\{.*?\}', re.DOTALL)
    
    # Busca el JSON en la cadena
    match = json_pattern.search(string)
    
    if match:
        json_str = match.group(0)
        try:
            # Intenta convertir la cadena JSON a un objeto Python
            json_obj = json.loads(json_str)
            return json_obj
        except json.JSONDecodeError:
            raise ValueError("La cadena contiene un JSON no válido")
    else:
        raise ValueError("No se encontró un JSON en la cadena proporcionada")

In [ ]:
try:
    extracted_json = extract_json_from_string(res['choices'][0]['message']['content'])
    print(extracted_json)
except ValueError as e:
    print(e)

In [ ]:
df = pd.DataFrame([extracted_json])